# Lab No 2: Data Manipulation and Working with Web Services

## 2 Challenges

# Challenge No 1

In [1]:
# Here is the code for challenge 2, Lab 2

#Uses a Dictionary to create a df with 4 columns and 150 rows 
import numpy as np
import pandas as pd


col1 = np.random.randint(2000, 2020, 150)
#column with 150 rows with random variables between 2000 and 2020

col2 = np.random.randint(20, 100, 150) 
#column with 150 rows with random variables between 20 and 100

col3 = np.random.randint(0, 10, 150)
#column with 150 rows with random variables between 0 and 10

col4 = np.random.randint(100, 300, 150) 
#column with 150 rows with random variables between 100 and 300

# np.random.randint generates random numbers to fill the defined array

#create dataframe with column headers a, b, c, d
df = pd.DataFrame({"a": col1, "b": col2, "c": col3, "d": col4}) 

df.head()

,a,b,c,d
0,2008,56,9,243
1,2015,96,4,194
2,2015,43,6,280
3,2005,83,9,256
4,2005,33,8,151


In [2]:
subset_df = df.loc[0:29, 'a':'c']
#create a subset of the data using .loc where the rows are between 0 and 29 and columns
#are between a and c

In [3]:
filtered_df = df.loc[(df['b'] > 50) & (df['c'] == 4)]
filtered_df.head()
# filter data so that it only includes rows where the value of b is greater than 50

,a,b,c,d
1,2015,96,4,194
7,2017,84,4,199
10,2013,63,4,288
25,2000,94,4,141
31,2019,86,4,287


In [5]:
#mean of all columns 
print(df.mean())

#standard deviation of all columns
print(df.std())


a    2009.740000
b      61.346667
c       4.573333
d     195.426667
dtype: float64
a     5.622856
b    24.450741
c     2.831621
d    59.298058
dtype: float64


In [16]:
#using group_by 

group = df.groupby('a')['c'].sum()
group

a
2000    30
2001    20
2002    37
2003    24
2004    37
2005    62
2006    71
2007    49
2008    47
2009    30
2010    11
2011    28
2012     6
2013    19
2014    39
2015    34
2016    21
2017    36
2018    48
2019    37
Name: c, dtype: int64

# Challenge No 2
## Part 1

In [20]:
# Here is the code for challenge 2, Lab 2

import requests
import pandas as pd
import geopandas as gpd


# the following code is taken from the example and is the preprocessing for the bikes data
url_bikes = "https://api.glasgow.gov.uk/mobility/v1/get_rentals?startDate=2022-05-01&endDate=2023-05-01"
# Making the query to the web server, using the Get method from the requests library 
response = requests.get(url_bikes)
response

<Response [200]>

In [ ]:
data = response.json()
data

In [ ]:
rental_data = data['data']
rental_data

In [23]:
rental_pd = pd.DataFrame(rental_data)

clean_rental_pd = rental_pd.dropna(subset=['startPlaceLat', 'startPlaceLong', 'endPlaceLat','endPlaceLong'])


In [27]:
gdf_bikes_end = gpd.GeoDataFrame(clean_rental_pd, geometry=gpd.points_from_xy(clean_rental_pd['endPlaceLong'], clean_rental_pd['endPlaceLat']))


In [28]:
# checking for any nan values in the coordinates 

nan_in_column_Lat = rental_pd['endPlaceLat'].isna().any()
nan_in_column_Long = rental_pd['endPlaceLong'].isna().any()

print(nan_in_column_Lat,nan_in_column_Lat)

True True


In [29]:
# remove nan values from the dataset and check to see how many have been dropped and the datatype of columns

clean_rental_pd = rental_pd.dropna(subset=['endPlaceLat','endPlaceLong'])
clean_rental_pd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9967 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   created                  9967 non-null   object 
 1   updated                  9967 non-null   object 
 2   cityId                   9967 non-null   object 
 3   bikeId                   9967 non-null   object 
 4   startDate                9967 non-null   object 
 5   startPlaceId             9967 non-null   object 
 6   startPlaceCityId         9967 non-null   object 
 7   startPlaceName           9967 non-null   object 
 8   startPlaceStationNumber  9559 non-null   float64
 9   startPlaceLat            9940 non-null   float64
 10  startPlaceLong           9940 non-null   float64
 11  startChannelId           0 non-null      object 
 12  endDate                  9967 non-null   object 
 13  endPlaceId               9967 non-null   object 
 14  endPlaceCityId           9967

In [30]:
# convert to a gdf using geopandas and using the coordinates to create a point geometry
gdf_bikes_end = gpd.GeoDataFrame(clean_rental_pd, geometry=gpd.points_from_xy(clean_rental_pd['endPlaceLong'], clean_rental_pd['endPlaceLat']))

In [31]:
# convert to a gdf using geopandas and using the coordinates to create a point geometry
gdf_bikes_end = gpd.GeoDataFrame(clean_rental_pd, geometry=gpd.points_from_xy(clean_rental_pd['endPlaceLong'], clean_rental_pd['endPlaceLat']))

In [32]:
# filter unwanted columns from the gdf
keep_cols = [
    "endDate",
    "endPlaceId",
    "endPlaceName",
    "durationSeconds",
    "isInvalid",
    "price",
    "isEbike",
    "endPlaceLat",
    "endPlaceLong",
    "geometry",
]
gdf_bikes_end = gdf_bikes_end[keep_cols]
gdf_bikes_end.head()

,endDate,endPlaceId,endPlaceName,durationSeconds,isInvalid,price,isEbike,endPlaceLat,endPlaceLong,geometry
0,2022-05-01T00:02:36Z,264299,ELECTRIC - Broomielaw,56,False,2.0,True,55.856600,-4.263521,POINT (-4.26352 55.85660)
1,2022-05-01T00:22:34Z,3812776,Alexandra Park (south entrance) Alexandra Para...,1097,False,1.0,False,55.863128,-4.210282,POINT (-4.21028 55.86313)
2,2022-05-01T00:10:08Z,264302,Botanic Gardens - ELECTRIC,344,False,1.0,False,55.878278,-4.288487,POINT (-4.28849 55.87828)
3,2022-05-01T02:57:49Z,349455,ELECTRIC - Cessnock Subway Station,10400,False,6.0,False,55.851918,-4.294490,POINT (-4.29449 55.85192)
4,2022-05-01T01:21:31Z,264295,ELECTRIC - St Enoch Square,4611,False,3.0,False,55.856829,-4.255292,POINT (-4.25529 55.85683)


In [33]:
# update to more appropriate data types for each of the columns for clustering

gdf_bikes_end.endPlaceId = gdf_bikes_end.endPlaceId.astype(int)
gdf_bikes_end.endPlaceName = gdf_bikes_end.endPlaceName.astype(str)
gdf_bikes_end['endDate'] = pd.to_datetime(gdf_bikes_end['endDate'], format='%Y-%m-%dT%H:%M:%SZ')

gdf_bikes_end.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 9967 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   endDate          9967 non-null   datetime64[ns]
 1   endPlaceId       9967 non-null   int64         
 2   endPlaceName     9967 non-null   object        
 3   durationSeconds  9967 non-null   int64         
 4   isInvalid        9967 non-null   bool          
 5   price            9967 non-null   float64       
 6   isEbike          9967 non-null   bool          
 7   endPlaceLat      9967 non-null   float64       
 8   endPlaceLong     9967 non-null   float64       
 9   geometry         9967 non-null   geometry      
dtypes: bool(2), datetime64[ns](1), float64(3), geometry(1), int64(2), object(1)
memory usage: 720.3+ KB


In [34]:
# clustering step: set for 4 clusters using the gdf_bikes_end gdf
# This establishes 4 clusters 

from sklearn.cluster import KMeans
num_clusters = 4

kmeans_collection = KMeans(n_clusters=num_clusters, random_state=42)
gdf_bikes_end['kmeans_cluster'] = kmeans_collection.fit_predict(gdf_bikes_end[['endPlaceLong', 'endPlaceLat']])

In [35]:
# plotting the clusters using leafmap

import leafmap

end_map = leafmap.Map(center=(55.860166, -4.257505),
                zoom=12,
                draw_control=False,
                measure_control=False,
                fullscreen_control=False,
                attribution_control=True,
                   
               )

end_map.add_basemap("CartoDB.Positron")
end_map.add_data(
    gdf_bikes_end,
    column='kmeans_cluster',
    legend_title='Clusters',
    cmap='Set2',
    k=4,
)

#Ploting the map
end_map

Map(center=[55.860166, -4.257505], controls=(AttributionControl(options=['position', 'prefix'], position='bott…